# Building Models with PyTorch
## `torch.nn.Module` and `torch.nn.Parameter`

Here, we'll discuss some of the tools PyTorch makes available for building deep learning networks.

Except for `Parameter`, the classes we disciuss in this video are all subclasses of `torch.nn.Module`. This is the PyTorch base class meant to encapsulate behaviours specific to PyTorch Models and their components.

One important behaviour of `torch.nn.Module` is registering parameters. If a particular `Module` subclass has learning weights, these weights are expressed as instances of `torch.nn.Parameter`. The `Parameter` class is a subclass of `torch.Tensor`, with the special behaviour that when they are assinged as attributes of a `Module`, they are added to the list of that module's parameters. These parameters may be accessed through the `parameters()` method on the `Module` class.

As a simple example, here's a very simple model with two linear layers and an activation function. We'll create an instance of it and task it to report on it's parameters:

In [1]:
import torch

class TinyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear (200, 10)
        self.softmax = torch.nn.Softmax()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    
tinymodel = TinyModel()

print ('The model:')
print (tinymodel)

print ('\n\nJust one layer:')
print (tinymodel.linear2)

print ('\n\nModel params:')
for param in tinymodel.parameters():
    print (param)
    
print ('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print (param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0490,  0.0928,  0.0199,  ..., -0.0030,  0.0403,  0.0134],
        [ 0.0431, -0.0076, -0.0022,  ..., -0.0435, -0.0585,  0.0077],
        [ 0.0056, -0.0066,  0.0319,  ...,  0.0839, -0.0310,  0.0756],
        ...,
        [ 0.0273,  0.0088, -0.0559,  ..., -0.0939, -0.0767,  0.0085],
        [-0.0120, -0.0971,  0.0538,  ..., -0.0601, -0.0466, -0.0634],
        [-0.0649, -0.0885,  0.0961,  ..., -0.0604,  0.0865,  0.0042]],
       requires_grad=True)
Parameter containing:
tensor([-3.0698e-02, -8.7677e-03,  6.3464e-02,  8.0533e-02, -8.5202e-02,
         4.7646e-02, -4.6430e-02, -3.8487e-02, -2.8487e-02, -6.1240e-03,
        -9.1468e-02, -8.6296e-02,  3.8921e-02,  3.4473

This shows the fundamental structure of a PyTorch model: there is an `__init__()` method that defines the layers and other components of a model, and a `forward()` method where the computation gets done. Note that we can print the model, or any of its submodules to learn about its structure.

## Common Layer Types

### Linear Layers
The most basic type of a neural network is a _linear_ or _fully conencted_ layer. This is a layer where every input influences every output of the layer to a degree specified by the layer's weights. If a model has _m_ inputs, and _n_ outputs, the weights will be an _m x n_ matrix. For example:

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print ('Input:')
print (x)

print ('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print (param)
    
y = lin(x)
print ('\n\nOutput:')
print (y)

Input:
tensor([[0.4231, 0.8234, 0.6379]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.2999,  0.5121, -0.2398],
        [ 0.3709, -0.2685, -0.0796]], requires_grad=True)
Parameter containing:
tensor([0.3193, 0.0170], requires_grad=True)


Output:
tensor([[ 0.7148, -0.0980]], grad_fn=<AddmmBackward0>)


If we do the matrix multiplication of `x` by the linear layer's weights, and add the biases, we get the output vector `y`.

One other important feature of note: When we checked the weights of our layer with `lin.weight`, it reported itself as a `Parameter` (which is a subclass of `Tensor`), and let us know that it's tracking gradients with autograd. This is a default behavious for `Parameter` that deffiers from `Tensor`.

Linear layers are used widely in deep learning models. One of the most common places we'll see them is in classifier models, which will usually have one or more linear layers at the end where the last layer will have _n_ outputs, where _n_ is the number of classes the classifier addresses.

### Convolutional Layers
_Convolutional_ layers are built to handle data with high degree of spatial correlation. They are very commonly used in computer vision, where they detect close groupings of features which they compose into higher-level features. They pop up in other contexts too - for example, in NLP applications, where a word's immediate context (that is, the other words nearby in the sequence) can affect the meaning of the sentence.

We saw convolutional layers in action in LeNet5 before:

In [3]:
import torch.functional as F

class LeNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.con1 = torch.nn.Conv2d(1, 6, 5)
        self.con2 = torch.nn.Conv2d(6, 16, 3)
        self.fc1 = torch.nn.Linear(16*6*6, 120) # 6*6 frm image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

Let's breakd down the code cell above. Starting with `conv1`:
- LeNet5 is meant to take in a 1x32x32 black & white image. **The first argument to a convolutional layer's constructor is the number of input channels.** Here, it is 1. If we were building this model to look at 3-colour channels, it would be 3.
- A convolutional layer is liek a window that scans over the image, looking for a pattern it recognises. These patterns are called _features_, and one of the parameters of a convolutional layer is teh number of features we would like to learn. **This is the second argument to the constructor; the number of output features.** Here, we're asking our layer to learn 6 features.
- Just above, we likened the convolutional layer to a window - but how big is the window? **The third argument is the window or kernel size.** Here, the "5" means we've chosen a 5x5 kernel. (If we want a kernel with height different from width, we can specify a tuple for this argument - e.g., `(3, 5)` to get a 3x5 convolution kernel.)

The output of a convolutional layer is teh _activation map_ - a spatial representation of the presence of features in the input tensor. `conv1` will give us an output tensor of 6x28x28; 6 is the number of features, and 28 is the height and width of our map. (The 28 comes from the fact that when scanning a 5-pixel window over a 32-pixel row, there are only 28 valid positions.)

We then pass the output of the convolution through a ReLU activation function (more on activation functions later), then through a max pooling layer. The max pooling layer takes features near each other int eh activation map, and groups them together. It does this by reducing the tensor, merging every 2x2 group of cells in the output into a single cell, and assigning that cell the maximum of the 4 cells that went into it. This gives us a lower-resolution version of the activation map, with dimensions 6x14x14.

Our next convolutional layer, `conv2`, expects 6 input channels (corresponding to the 6 features sought by the first layer), has 16 output channels, and a 3x3 kernel. It puts out a 16x12x12 activation map, which is again reduced by a max pooling layer to 16x6x6. Prior to passing this output to the linear layers, it is reshaped to a 16 \* 6 \* 6 = 576-element vector for consumption by the next layer.
There are convolutional layers addressing 1D, 2D, and 3D tensors. There are also many more optional aguments for a conv layer constructor, including stride length (e.g., only scanning every second or every third position) in the input, padding (so we can scan out to the edges of the input), and more. See the [documentation](https://pytorch.org/docs/stable/nn.html#convolution-layers) for more information.

### Recurrent Layers

_Recurrent Neural Networks_ (or RNNs) are used for sequential data - anything from time-series measurements from a scientific instrument to natural language sentences to DNA neucleotides. An RNN does this by maintaining a _hidden state_ that acts as a sort of memory for what it has seen in the sequence so far.

The internal structure of an RNN layer - or its variants, the LSTM (long short-term memory) and GRU (gated recurrent unit) - is moderately complex and beyond the scope of this document, but we will see what one looks like in action with an LSTM-based part-of-speech tagger (a type of classifier that tells if a word is a noun, verb, etc.):

In [4]:
class LSTMTagger(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that maps gfrom hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scrores = F.log_softmax(tag_space, dim=1)
        return tag_scores

The contructor has 4 arguments:
- `vocab_size` is the number of words in the input vocabulary. Each word is a one-hot vector (or unit vector) in a `vocab_size`-dimensional space.
- `tagset_size` is the number of tags in teh output set.
- `embedding_dim` is the size of the _embedding_ space for the vocabulary. An embedding maps a vocabulary onto a low-dimensional space, where words with similar meanings are closer together in the space.
- `hidden_dim` is the size of the LSTM's memory.

The input will be a sentence with the words represented as indices of one-hot vectors. The mebedding layer will then map these down to an `embedding_dim`-dimensional space. The LSTM takes this sequence of embeddings and iterates over it. Fielding an output vector of length `hidden_dim`. The final linear layer acts as a classifier; applying `log_softmax()` to the output of the final layer convers the output into a normalized set of estimated probabilities that a given word maps to a given tag.

To see the network in action, check out the [Sequence Models and LSTM Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) tutorial on pytorch.org.

### Transformers

_Transformers_ are multi-purpose networks that have taken over the state-of-the-artin NLP with models like BERT. A discussion of transformer architecture is beyond the scope of this document, but PyTorch has a `Transformer` class that allows us to define the overall parameters of a transformer model - the number of attention heads, the number of encoder & decoder layers, dropout and activation functions, etc. (We can even build the BERT model from this single class, with the right parameters!) The `Torch.nn.Transformer` class also has classes to encapsulate the individual components (`TransformerEncoderLayer`, `TransformerDecoderLayer`). For details, check the [documentation](https://pytorch.org/docs/stable/nn.html#transformer-layers) on transformer classes, and the relevant [tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html) on pytorch.org.

## Other Layers and Functions
### Data Manipulation Layers
There are other layer types that perform important functions in models, but don't participate in the learning process themselves.

**Max pooling** (and its twin, min pooling) reduce a tensor by combining cells, and assigning the maximm value of the input cells to the output cell (we saw this). For example:

In [5]:
my_tensor = torch.rand(1, 6, 6)
print (my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print (maxpool_layer(my_tensor))

tensor([[[0.0549, 0.8156, 0.8104, 0.6509, 0.6754, 0.6188],
         [0.6831, 0.8625, 0.1405, 0.2928, 0.5995, 0.3090],
         [0.5759, 0.0909, 0.2288, 0.7346, 0.1516, 0.1940],
         [0.3257, 0.5399, 0.1833, 0.3616, 0.2169, 0.3505],
         [0.2414, 0.7608, 0.0111, 0.7858, 0.0402, 0.8128],
         [0.5773, 0.4504, 0.2304, 0.6537, 0.2951, 0.6097]]])
tensor([[[0.8625, 0.7346],
         [0.7608, 0.8128]]])


Looking closely at the values above, we see that each of the values in the maxpooled output is the maximum value of each quadrant of the 6x6 input.

**Normalization layes** re-center and normalize the output of one layer before feeding it to another. Centering and scaling the intermediate tensors has a number of beneficial effects, such as letting us use higher learning rates without exploding/vanishing gradients.

In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print (my_tensor)

print (my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print (normed_tensor)

print (normed_tensor.mean())

tensor([[[15.0707, 24.1291, 16.2062, 24.8783],
         [ 6.7288, 22.1057, 11.9832,  7.8761],
         [ 9.5640,  9.3455, 17.0561, 23.4175],
         [ 5.2270,  8.3228, 12.3505, 24.0879]]])
tensor(14.8968)
tensor([[[-1.1215,  0.9101, -0.8668,  1.0782],
         [-0.8987,  1.6395, -0.0314, -0.7094],
         [-0.9041, -0.9415,  0.3784,  1.4673],
         [-1.0164, -0.5836, -0.0205,  1.6205]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-3.7253e-08, grad_fn=<MeanBackward0>)


Running the cell above, we've added a large scaling factor and offset to an input tensor; we see the input tensor's `mean()` is somewhere around 15. After running it through the normalization layer, we see that the values are smaller, and grouped around zero - in fact, the mean should be very small (>1e-8).

This is beneficial because many activation functions (discussed below) have their strongest gradients near 0, but sometimes suffer from vanishing or exploding gradients for inputs that drive them far away from zero. Keeping the data centered around the area of steepest gradient will tend to mean faster, better learning and higher feasible learning rates.

**Dropout layers** are a tool for encouraging _sparse representations_ in our model - that is, pushing it to do inference with less data.

Dropout layers work by randomly setting parts of the input tensor off _during training_ - dropout layers are always turned off for inference. This forces the model to learn against this masked or reduced dataset. For example:

In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print (dropout(my_tensor))
print (dropout(my_tensor))

tensor([[[0.0000, 1.6511, 0.3868, 0.0000],
         [0.5219, 1.1435, 0.0000, 1.3716],
         [0.0000, 0.0000, 0.9656, 0.5082],
         [0.2845, 0.0773, 0.2342, 0.0000]]])
tensor([[[1.2364, 0.0000, 0.0000, 0.1478],
         [0.5219, 1.1435, 0.0000, 1.3716],
         [0.0000, 1.2196, 0.9656, 0.0000],
         [0.2845, 0.0773, 0.2342, 0.0000]]])


Above, we see the effect of dropout on a sample tensor. We can use the optional `p` argument to set the probability of an individual weight dropping out; is we don't the default is 0.5.

### Activation Functions
Activation functions make deep learning possible. A neural network is really a program - with many parameters - that _simulates mathematical function_. If all we did was multiply tensors by layer weights repeatedly, we could only simulate _linear functions_; further, there would be no point to having many layers, sa the whole network would reduce to a single matrix multiplication. Inserting _non-linearity_ in the form of activation functions between layers is what allows a deep learning model to simulate any function, rather than just linear ones.

`torch.nn.Module` has objects encapsulating all of the major activation functions including ReLU and its many variants, Tanh, Hardtanh, sigmoid and more. It also includes other functions, such as Softmax, that are most useful at eh output stage of a model.

### Loss Functions
Loss functions tell us how far a model's prediction is from the correct answer. PyTorch contains a variety of loss functions, including common MSE (mean sqiared error = L2 norm), Cross Entropy Loss and Negative Likelihood Loss (useful for classifiers), and others.